# Project - New car registrations in Europe
DATA PROCESSING NOTEBOOK

### 0. Necesary libraries

In [1]:
import pandas as pd
import numpy as np

### 1. Finding specific names for countries and motor energy types

Downloading the frames

In [2]:
names = pd.read_html('https://ec.europa.eu/eurostat/statistics-explained/index.php?title=Glossary:Country_codes')

In [3]:
names

[          0     1        2     3            4     5         6     7
 0   Belgium  (BE)   Greece  (EL)    Lithuania  (LT)  Portugal  (PT)
 1  Bulgaria  (BG)    Spain  (ES)   Luxembourg  (LU)   Romania  (RO)
 2   Czechia  (CZ)   France  (FR)      Hungary  (HU)  Slovenia  (SI)
 3   Denmark  (DK)  Croatia  (HR)        Malta  (MT)  Slovakia  (SK)
 4   Germany  (DE)    Italy  (IT)  Netherlands  (NL)   Finland  (FI)
 5   Estonia  (EE)   Cyprus  (CY)      Austria  (AT)    Sweden  (SE)
 6   Ireland  (IE)   Latvia  (LV)       Poland  (PL)       NaN   NaN,
                0     1   2            3     4
 0        Iceland  (IS) NaN       Norway  (NO)
 1  Liechtenstein  (LI) NaN  Switzerland  (CH),
                 0     1   2
 0  United Kingdom  (UK) NaN,
                         0     1
 0  Bosnia and Herzegovina  (BA)
 1              Montenegro  (ME)
 2                 Moldova  (MD)
 3         North Macedonia  (MK)
 4                 Albania  (AL)
 5                  Serbia  (RS)
 6             

Transforming the 1st frame

In [4]:
names0_1 = names[0][[2, 3]]
names0_1.columns = [0, 1]
names0_2 = names[0][[4, 5]]
names0_2.columns = [0, 1]
names0_3 = names[0][[6, 7]]
names0_3.columns = [0, 1]

names0_all = pd.concat([names[0][[0, 1]], names0_1, names0_2, names0_3], axis = 0, ignore_index = True)
names0_all.dropna(inplace = True)

Transforming the 2nd frame

In [5]:
names1_1 = names[1][[3, 4]]
names1_1.columns = [0, 1]

names1_all = pd.concat([names[1][[0, 1]], names1_1], axis = 0, ignore_index = True)

Creating one frame with all the codes

In [6]:
codes = pd.concat([names0_all, names1_all, names[2][[0, 1]], names[3], names[4], names[5]], axis = 0, ignore_index = True)

Transforming the codes frame

In [7]:
codes[0] = codes[0].str.strip('*')
codes[1] = codes[1].str.strip('()[1]')
codes.columns = ['Country', 'Country code']
codes.set_index('Country code', inplace=True)
codes.replace('Türkiye', 'Turkiye', inplace=True)

In [8]:
codes.head()

,Country
Country code,
BE,Belgium
BG,Bulgaria
CZ,Czechia
DK,Denmark
DE,Germany


Finding the motor type name

In [9]:
motor_types = {'PET': 'Petrol', 'LPG': 'LPG', 'DIE': 'Diesel', 'GAS': 'Natural gas', 'ELC': 'Electricity',
               'ALT': 'Alternative energy', 'PET_X_HYB': 'Petrol (excluding hybrids)', 'ELC_PET_HYB': 'Hybrid electric-petrol',
               'ELC_PET_PI': 'Plug-in hybrid electric-petrol', 'DIE_X_HYB': 'Diesel (excluding hybrids)',
               'ELC_DIE_HYB': 'Hybrid electric-diesel', 'ELC_DIE_PI': 'Plug-in hybrid electric-diesel', 
               'HYD_FCELL': 'Hydrogen and fuel cells', 'BIOETH': 'Bioethanol', 'BIODIE': 'Biodiesel', 'BIFUEL': 'Bi-fuel', 'OTH': 'Other'}

### 2. Frame with the info about new car registrations

In [10]:
new_cars = pd.read_csv('https://ec.europa.eu/eurostat/estat-navtree-portlet-prod/BulkDownloadListing?file=data/road_eqr_carpda.tsv.gz', 
                       delimiter = '\t')

In [11]:
new_cars.head()

,"unit,mot_nrg,geo\time",2022,2021,2020,2019,2018,2017,2016,2015,2014,2013
0,"NR,ALT,AL",4114,5703,4935,3757,:,:,:,:,:,:
1,"NR,ALT,AT",34242,33466,16393,9841,7406,5868,4317,2389,2074,1285
2,"NR,ALT,BA",300,229,73,137,41,136,141,49,46,39
3,"NR,ALT,BE",40656,25867,18920,12525,7931,5418,4413,2147,2234,841
4,"NR,ALT,BG",:,23,:,:,:,:,:,:,:,:


#### Transforming the new cars table

Extracting the data from the first loaded column

In [12]:
motor_country_columns = new_cars.iloc[:,0].str.split(',', expand = True)
new_cars[['Motor energy', 'Country']] = motor_country_columns.iloc[:,1:3]
new_cars.drop(columns = new_cars.columns[0], inplace = True)

Correcting the columns names

In [13]:
new_cars.columns = new_cars.columns.str.strip()

Mapping the 'Motor energy' and 'Country' columns with specified values

In [14]:
new_cars = new_cars.merge(codes, left_on = 'Country', right_on = 'Country code', how = 'left', suffixes = ('_old', ''))

new_cars.drop(columns = 'Country_old', inplace = True)
new_cars['Motor energy'] = new_cars['Motor energy'].map(motor_types)

Creating a multiindex

In [15]:
new_cars.set_index(['Country', 'Motor energy'], inplace = True)
new_cars.sort_index(inplace = True)
new_cars.replace(': ', '0', inplace = True)
new_cars.replace('', '0', inplace = True)

Removing special characters

In [16]:
for year in range(2014, 2019):
    new_cars.at[('Austria', 'Bioethanol'), str(year)] = '0'
    new_cars.at[('Austria', 'Biodiesel'), str(year)] = '0'
new_cars.at[('Austria', 'LPG'), '2017'] = '0'

In [17]:
for col in new_cars.columns:
    new_cars[col] = new_cars[col].str.strip(' :nzbsed')
new_cars.fillna('0', inplace = True)

Setting proper types

In [18]:
for year in new_cars.columns:
    new_cars[year] = new_cars[year].astype(str)
    new_cars[year] = new_cars[year].astype(int)

Final, representative for of the table

In [19]:
new_cars.head()

2022   2021   2020   2019  2018  2017   
Country Motor energy                                                         
Albania Alternative energy           4114   5703   4935   3757     0     0  \
        Bi-fuel                      3432   5310   4551      0     0     0   
        Diesel                      36523  42914  32953  33064     0     0   
        Diesel (excluding hybrids)  36189  42736  32906  33041     0     0   
        Electricity                   624    262    239     24     0     0   

                                    2016  2015  2014  2013  
Country Motor energy                                        
Albania Alternative energy             0     0     0     0  
        Bi-fuel                        0     0     0     0  
        Diesel                         0     0     0     0  
        Diesel (excluding hybrids)     0     0     0     0  
        Electricity                    0     0     0     0

#### Transforming for more convenient visulaisation

Groupping motor types and merging rows

In [20]:
for country in new_cars.index.get_level_values(0).unique():
    
    row_hybrid = np.zeros(10)
    row_other = np.zeros(10)
    for motor in new_cars.loc[country].index:
        if motor in ['Hybrid electric-diesel', 'Hybrid electric-petrol']:
            row_hybrid += np.array(new_cars.loc[(country, motor)].to_list())
        elif motor in ['Diesel', 'Petrol', 'LPG', 'Electricity', 'Alternative energy', 'Petrol (excluding hybrids)', 'Diesel (excluding hybrids)']:
            pass
        else:
            row_other += np.array(new_cars.loc[(country, motor)].to_list())

    new_cars.loc[(country, 'Hybrid'), :] = {year:value for year,value in zip(new_cars.columns, row_hybrid)}
    new_cars.loc[(country, 'Other'), :] = {year:value for year,value in zip(new_cars.columns, row_other)}

Deleting and renaming rows

In [21]:
new_cars.drop(index=['Alternative energy', 'Bi-fuel', 'Biodiesel', 'Bioethanol', 'Petrol',
                     'Hybrid electric-diesel', 'Hybrid electric-petrol', 'Diesel', 'Natural gas',
                     'Hydrogen and fuel cells', 'Plug-in hybrid electric-diesel', 'Plug-in hybrid electric-petrol'],
              level = 1, inplace=True)

new_cars.rename(index={'Diesel (excluding hybrids)':'Diesel', 'Petrol (excluding hybrids)':'Petrol'}, inplace=True)
new_cars.sort_index(inplace=True)

Setting proper types

In [22]:
for year in new_cars.columns:
    new_cars[year] = new_cars[year].astype(int)

Changing the shape

In [23]:
new_cars = new_cars.stack().to_frame()
new_cars.columns = ['New cars']
new_cars.index.rename({None : 'Year'}, inplace = True)

Final form, ready to be exported

In [24]:
new_cars.head()

New cars
Country Motor energy Year          
Albania Diesel       2022     36189
                     2021     42736
                     2020     32906
                     2019     33041
                     2018         0

### 3. Frame with info about car stocks

In [25]:
cars = pd.read_csv('https://ec.europa.eu/eurostat/estat-navtree-portlet-prod/BulkDownloadListing?file=data/road_eqs_carpda.tsv.gz',
                  delimiter = '\t')

In [26]:
cars.head()

,"unit,mot_nrg,geo\time",2022,2021,2020,2019,2018,2017,2016,2015,2014,2013
0,"NR,ALT,AL",59347,52846,43675,35881,28994,21409,14544,7562,4373,:
1,"NR,ALT,AT",115799,82381,50615,35642,26732,20180,14459,10125,7931,5972
2,"NR,ALT,BA",36259,37469,39366,38302,39603,34579,8350,3914,25547,22436
3,"NR,ALT,BE",126338,93670,74124,62694,54170,47324,44061,42575,18848,18859
4,"NR,ALT,BG",400014 s,374094 s,352488 s,:,:,:,:,:,:,:


Data preprocessing

In [27]:
motor_country_columns = cars.iloc[:,0].str.split(',', expand = True)
cars[['Motor energy', 'Country']] = motor_country_columns.iloc[:,1:3]
cars.drop(columns = cars.columns[0], inplace = True)

In [28]:
cars.columns = cars.columns.str.strip()

In [29]:
cars = cars.merge(codes, left_on = 'Country', right_on = 'Country code', how = 'left', suffixes = ('_old', ''))
cars.drop(columns = 'Country_old', inplace = True)

In [30]:
cars['Motor energy'] = cars['Motor energy'].map(motor_types)

In [31]:
cars.dropna(subset = ['Motor energy', 'Country'], inplace = True)

In [32]:
cars.set_index(['Country', 'Motor energy'], inplace = True)
cars.sort_index(inplace = True)

In [33]:
cars.replace(': ', '0', inplace = True)
cars.replace('', '0', inplace = True)

In [34]:
for year in range(2013, 2019):
    cars.at[('Austria', 'Bioethanol'), str(year)] = '0'
    cars.at[('Austria', 'Biodiesel'), str(year)] = '0'
cars.at[('Bosnia and Herzegovina', 'Plug-in hybrid electric-petrol'), '2018'] = 0
cars.at[('Croatia', 'Bioethanol'), '2015'] = 0
cars.at[('Croatia', 'Natural gas'), '2015'] = 0
cars.at[('Croatia', 'Hydrogen and fuel cells'), '2015'] = 0
cars.at[('Romania', 'Biodiesel'), '2019'] = 0
cars.at[('Romania', 'Bioethanol'), '2019'] = 0
cars.at[('Romania', 'Hydrogen and fuel cells'), '2019'] = 0

In [35]:
for col in cars.columns:
    cars[col] = cars[col].str.strip(' :nzbsed.')
cars.fillna('0', inplace = True)

In [36]:
cars['2013'].replace({'45.290': '45290', '90.450': '90450', '0.030': '30', '15.250': '15250', 
                      '13.920': '13920', '29': '29000', '351.650': '351650'}, inplace = True)

In [37]:
for year in cars.columns:
    cars[year] = cars[year].astype(int)

In [38]:
cars.head()

2022    2021    2020    2019    2018   
Country Motor energy                                                         
Albania Alternative energy           59347   52846   43675   35881   28994  \
        Diesel                      471551  437038  396887  366379  335240   
        Diesel (excluding hybrids)  470930  436756  396790  366330  335215   
        Electricity                   1245     624     362     124     102   
        Hybrid electric-diesel         619     282      97      49      25   

                                      2017    2016    2015    2014  2013  
Country Motor energy                                                      
Albania Alternative energy           21409   14544    7562    4373     0  
        Diesel                      300905  317053  292461  272421     0  
        Diesel (excluding hybrids)  300891  317045  292457  272420     0  
        Electricity                     48      13       0       0     0  
        Hybrid electric-diesel          14       8       4       1     0

In [39]:
cars['Cars'] = 0

for country, motor in cars.index:
    for year in cars.columns[::-1]:
        c = int(cars.loc[(country, motor), year])
        if c > 0:
            cars.at[(country, motor), 'Cars'] = c

In [40]:
cars.drop(columns=[str(year) for year in range(2013, 2023)], inplace=True)

In [41]:
cars['Cars'] = cars['Cars'].astype(int)

In [42]:
for country in cars.index.get_level_values(0).unique():
    
    count_hybrid = 0
    count_other = 0
    for motor in cars.loc[country].index:
        if motor in ['Hybrid electric-diesel', 'Hybrid electric-petrol']:
            count_hybrid += cars.loc[(country, motor), 'Cars']
        elif motor in ['Diesel', 'Petrol', 'LPG', 'Electricity', 'Alternative energy', 'Petrol (excluding hybrids)', 'Diesel (excluding hybrids)']:
            pass
        else:
            count_other += cars.loc[(country, motor), 'Cars']

    cars.loc[(country, 'Hybrid'), 'Cars'] = count_hybrid
    cars.loc[(country, 'Other'), 'Cars'] = count_other

In [43]:
cars.drop(index=['Alternative energy', 'Bi-fuel', 'Biodiesel', 'Bioethanol', 'Petrol',
                     'Hybrid electric-diesel', 'Hybrid electric-petrol', 'Diesel', 'Natural gas',
                     'Hydrogen and fuel cells', 'Plug-in hybrid electric-diesel', 'Plug-in hybrid electric-petrol'],
              level = 1, inplace=True)

cars.rename(index={'Diesel (excluding hybrids)':'Diesel', 'Petrol (excluding hybrids)':'Petrol'}, inplace=True)
cars.sort_index(inplace=True)

In [44]:
cars['Cars'] = cars['Cars'].astype(int)

In [45]:
cars.head()

Cars
Country Motor energy        
Albania Diesel        470930
        Electricity     1245
        Hybrid          2083
        LPG              396
        Other          57715

### 4. Frame with info about population

In [46]:
population = pd.read_csv('https://ec.europa.eu/eurostat/estat-navtree-portlet-prod/BulkDownloadListing?file=data/demo_pjan.tsv.gz',
                        delimiter = '\t')

In [47]:
population.head()

,"unit,age,sex,geo\time",2022,2021,2020,2019,2018,2017,2016,2015,2014,...,1969,1968,1967,1966,1965,1964,1963,1962,1961,1960
0,"NR,TOTAL,F,AD",:,:,:,37388,:,:,:,:,:,...,:,:,:,:,:,:,:,:,:,:
1,"NR,TOTAL,F,AL",1406532,1419759,1425342,1432833,1431715,1423050,1417141,1424597,1430827,...,:,:,:,:,:,:,:,:,:,:
2,"NR,TOTAL,F,AM",:,1565144,1562689,1563538,1564533,1567380,1569535,1571450,:,...,:,:,:,:,:,:,:,:,:,:
3,"NR,TOTAL,F,AT",4553444,4535712,4522292,4501742,4483749,4460424,4427918,4384529,4352447,...,3932691,3922359,3899799,3876559,3857760,3836415,3814191,3794130,3773097,3757167
4,"NR,TOTAL,F,AZ",5081846,5065288,5039100,4999053,4960058,4918771,4870002,4817181,4763571,...,:,:,:,:,:,:,:,:,:,:


In [48]:
country_column = population.iloc[:,0].str.split(',', expand = True)
population[['Age', 'Sex', 'Country']] = country_column.iloc[:,1:4]
population.drop(columns = population.columns[0], inplace = True)

In [49]:
population = population.merge(codes, left_on = 'Country', right_on = 'Country code', how = 'inner', suffixes = ('_old', ''))

In [50]:
population = population[(population['Age'] == 'TOTAL') & (population['Sex'] == 'T')]

In [51]:
population.drop(columns = ['Country_old', 'Age', 'Sex'], inplace = True)

In [52]:
population.columns = population.columns.str.strip()

In [53]:
population.set_index(['Country'], inplace = True)
population.sort_index(inplace = True)
population.replace(': ', 0, inplace = True)

In [54]:
for col in population.columns:
    population[col] = population[col].str.strip(' :nzbsedp')
population.fillna(0, inplace = True)

In [55]:
population.head()

,2022,2021,2020,2019,2018,2017,2016,2015,2014,2013,...,1969,1968,1967,1966,1965,1964,1963,1962,1961,1960
Country,,,,,,,,,,,,,,,,,,,,,
Albania,2793592,2829741,2845955,2862427,2870324,2876591,2875592,2885796,2892394,2897770,...,2052778,1991765,1939430,1889715,1839866,1788404,1736838,1685800,1633800,1583800
Armenia,0,2963251,2959694,2965269,2972732,2986151,2998577,3010598,0,0,...,0,0,0,0,0,0,0,0,0,0
Austria,8978929,8932664,8901064,8858775,8822267,8772865,8700471,8584926,8507786,8451860,...,7426968,7403837,7350159,7293973,7247804,7199798,7151824,7107904,7064693,7030385
Azerbaijan,10156366,10119133,10067108,9981457,9898085,9809981,9705643,9593038,9477119,9356483,...,0,0,0,0,0,0,0,0,0,0
Belarus,0,0,0,9475174,9491823,9504704,9498364,9480868,9468154,9463840,...,0,0,0,0,0,0,0,0,0,0


In [56]:
population['Population'] = 0
population.at['Kosovo', '2012'] = 0

for country in population.index:
    for year in population.columns[-2::-1]:
        pop = int(population.at[country, year])
        if pop > 0:
            population.at[country, 'Population'] = pop

In [57]:
population.drop(columns = ['{}'.format(i) for i in range(1960, 2023)], inplace = True)

In [58]:
population['Population'] = population['Population'].astype(int)

In [59]:
for country in population.index:
    if country not in cars.index.get_level_values(0).unique():
        population.drop(index=country, inplace=True)

In [60]:
population.head()

,Population
Country,
Albania,2793592
Austria,8978929
Belgium,11617623
Bosnia and Herzegovina,3839265
Bulgaria,6838937


### 5. Frame with info about GDP per capita

In [61]:
gdp = pd.read_csv('https://ec.europa.eu/eurostat/estat-navtree-portlet-prod/BulkDownloadListing?file=data/sdg_08_10.tsv.gz',
                 delimiter = '\t')

In [62]:
gdp.head()

,"unit,na_item,geo\time",2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,"CLV10_EUR_HAB,B1GQ,AL",1700,1850,1940,2060,2180,2310,2460,2630,2850 d,...,3260 d,3330,3410,3530,3670,3830,3920,3810 p,:,:
1,"CLV10_EUR_HAB,B1GQ,AT",31710,31990,32360,32520,33200,33710,34700,35870,36280,...,36180,36130,36140,36390,36980,37690,38070,35390,36740,38080
2,"CLV10_EUR_HAB,B1GQ,BE",29890,30120,30490,30680,31640,32200,32800,33760,33640,...,33490,33870,34360,34620,35040,35510,36110,34050,36230,37040 p
3,"CLV10_EUR_HAB,B1GQ,BG",2990,3210,3420,3620,3870,4170,4480,4800,5120,...,5390,5470,5700,5910,6120,6330,6630,6400,6950,7680 b
4,"CLV10_EUR_HAB,B1GQ,CH",51950,52510,52100,51700,52750,53880,55730,57400,58310,...,58650,59300,59600,60170,60420,61690,61950,60190,62950 p,64000 p


In [63]:
country_column = gdp.iloc[:,0].str.split(',', expand = True)
gdp[['Type', 'Country']] = country_column.iloc[:,[0, 2]]
gdp.drop(columns = gdp.columns[0], inplace = True)

In [64]:
gdp = gdp[gdp['Type'] == 'CLV10_EUR_HAB']

In [65]:
gdp = gdp.merge(codes, left_on = 'Country', right_on = 'Country code', how = 'inner', suffixes = ('_old', ''))

In [66]:
gdp.drop(columns = ['Country_old', 'Type'], inplace = True)

In [67]:
gdp.columns = gdp.columns.str.strip()

In [68]:
gdp.set_index(['Country'], inplace = True)
gdp.sort_index(inplace = True)
gdp.replace(': ', 0, inplace = True)

In [69]:
for year in range(2021, 2023):
    gdp.at['Albania', str(year)] = '0'
for year in range(2000, 2006):
    gdp.at['Montenegro', str(year)] = '0'
for year in range(2021, 2023):
    gdp.at['North Macedonia', str(year)] = '0'
for year in range(2020, 2023):
    gdp.at['United Kingdom', str(year)] = '0'

In [70]:
for col in gdp.columns:
    gdp[col] = gdp[col].str.strip(' epbsdnz')

In [71]:
gdp.head()

,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
Country,,,,,,,,,,,,,,,,,,,,,
Albania,1700,1850,1940,2060,2180,2310,2460,2630,2850,2960,...,3260,3330,3410,3530,3670,3830,3920,3810,0,0
Austria,31710,31990,32360,32520,33200,33710,34700,35870,36280,34830,...,36180,36130,36140,36390,36980,37690,38070,35390,36740,38080
Belgium,29890,30120,30490,30680,31640,32200,32800,33760,33640,32700,...,33490,33870,34360,34620,35040,35510,36110,34050,36230,37040
Bulgaria,2990,3210,3420,3620,3870,4170,4480,4800,5120,4970,...,5390,5470,5700,5910,6120,6330,6630,6400,6950,7680
Croatia,7940,8510,8990,9500,9890,10310,10830,11380,11610,10780,...,10480,10480,10810,11290,11800,12250,12740,11700,13610,14660


In [72]:
gdp['GDP'] = 0

for country in gdp.index:
    for year in gdp.columns:
        g = int(gdp.at[country, year])
        if g > 0:
            gdp.at[country, 'GDP'] = g

In [73]:
gdp.drop(columns = ['{}'.format(i) for i in range(2000, 2023)], inplace = True)

In [74]:
gdp['GDP'] = gdp['GDP'].astype(int)

In [75]:
for country in gdp.index:
    if country not in cars.index.get_level_values(0).unique():
        gdp.drop(index=country, inplace=True)

In [76]:
gdp.head()

,GDP
Country,
Albania,3810
Austria,38080
Belgium,37040
Bulgaria,7680
Croatia,14660


### 6. Exporting all the tables to csv files

In [77]:
for file in ['new_cars', 'cars', 'population', 'gdp']:
    eval(file).to_csv(f'{file}.csv', sep = ',', header = True, mode = 'w', decimal = '.', index = True)